In [ ]:
pip install accelerate

In [ ]:
pip install jiwer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
model_id = "Aditya02/Vistar_Marathi_Model"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, low_cpu_mem_usage=True,torch_dtype = torch_dtype
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [10]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    torch_dtype = torch_dtype,
    device=device
)

In [ ]:
directory = '/content/drive/MyDrive/common_voice_test'

file_names = []
for filename in os.listdir(directory):
  file_names.append(filename)
file_names.sort()
file_names.reverse()
file_names.pop(0)

In [ ]:
predicted_texts=[]
for file_name in file_names:
  file_path = os.path.join(directory,file_name)
  result = pipe(file_path)
  predicted_texts.append(result['text'])

In [ ]:
len(predicted_texts)

In [ ]:
reference_file = '/content/drive/MyDrive/common_voice_test/trans.txt'
reference_texts = []
with open(reference_file,'r', encoding='utf-8') as f:
    for line in f:
      text = line.strip().split('\t')
      reference_texts.append(text[1])

In [ ]:
len(reference_texts)

In [ ]:
errors = []
for i in range(len(predicted_texts)):
  error = jiwer.wer(reference_texts[i], predicted_texts[i])
  errors.append(error)

In [ ]:
len(errors)

In [ ]:
print(np.mean(errors))

0.22390942390942387